Aims:
- Do AR to fit model to time series and get features.  Trying frequency of oscillations for now, potential to expand to quality (height of peak of periodogram).
- Do it with Causton strains.  With potential to switch to the experiment with the new CEN.PK if I have time.
- Produce some plots for BYG201 (panel 6)

Specify file name and sampling period

In [2]:
#filename_prefix = './data/arin/Omero19979_'
filename_prefix = './data/arin/Omero20016_'
sampling_period = 5
remain = 0.8

%matplotlib

Using matplotlib backend: TkAgg


Main shebang

In [3]:
#!/usr/bin/env python3
import os

import numpy as np
import scipy as sp
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import sklearn.metrics
import igraph as ig

import pipeline.dataexport
import pipeline.dataimport
import pipeline.periodogram
import pipeline.score
import pipeline.tsman
import pipeline.vis

import featext.tsman
import featext.graph
#import featext.vis

#import catch22
#import leidenalg

def add_classicalAttr(cell, oversampling_factor = 1):
    """Computes classical periodogram and adds PdgramAttr attributes"""
    cell.flavin.classical.freqs, cell.flavin.classical.power = \
            pipeline.periodogram.classical(cell.time, cell.flavin.reading_processed,
                                oversampling_factor = oversampling_factor)

def add_bglsAttr(cell):
    """Computes BGLS and adds PdgramAttr attributes"""
    cell.flavin.bgls = pipeline.PdgramAttr()
    cell.flavin.bgls.label = 'Bayesian General Lomb-Scargle Periodogram'
    cell.flavin.bgls.power_label = 'Probability'
    err = np.ones(len(cell.flavin.reading_processed))*\
            np.sqrt(np.max(cell.flavin.reading_processed))
    cell.flavin.bgls.freqs, cell.flavin.bgls.power = \
            pipeline.periodogram.bgls(cell.time, cell.flavin.reading_processed, err,
                    plow = 30.0, phigh = 360.0, ofac = 5)

def add_autoregAttr(cell):
    """
    Computes autoregressive model-based periodogram and adds PdgramAttr
    attributes
    """
    cell.flavin.autoreg = pipeline.PdgramAttr()
    cell.flavin.autoreg.label = \
            'Autogressive Model-Based Periodogram (Jia & Grima, 2020)'
    cell.flavin.autoreg.power_label = 'Power'
    freq_npoints = 1000
    cell.flavin.autoreg.order, cell.flavin.autoreg.freqs, cell.flavin.autoreg.power = \
            pipeline.periodogram.autoreg(cell.time,
                                         cell.flavin.reading_processed,
                                         freq_npoints)

# FLAVIN: import data and process objects

# Import fluorescence info from CSVs
Dset_flavin = pipeline.dataimport.import_timeseries(
    filename_prefix+'flavin.csv', remain = remain)
# dummy so I get code to not complain; will be re-factored later
Dset_dcategory = [3] * len(Dset_flavin)
Dset_births = pipeline.dataimport.import_births(
    filename_prefix+'births.csv')

# Arranges information into DatasetAttr objects
Dset_data = pipeline.dataimport.CellAttr_from_datasets( \
        timeseries_df = Dset_flavin,
        categories_array = Dset_dcategory,
        births_df = Dset_births,
        sampling_pd = sampling_period)
Dset = pipeline.DatasetAttr(Dset_data)

# Add labels
strainlookup = pd.read_csv(filename_prefix+'strains.csv', \
                          index_col = 'position')
for ii, cell in enumerate(Dset.cells):
    cell.source = filename_prefix
    cell.medium.base = 'Delft'
    cell.medium.nutrients = {'glucose': 10}

    cell.strain = strainlookup.loc[cell.position].strain

    cell.flavin = pipeline.Fluo('flavin')
    cell.flavin.exposure = 60
    cell.flavin.reading = cell.y
    cell.flavin.category = Dset_dcategory[ii]


# mCherry: import data and process objects
try:
    Dset_mCherry_unsliced = pipeline.dataimport.import_timeseries(
        filename_prefix+'mCherry.csv', remain = remain)
    # restrict to cells with flavin readings
    idx_both = list(set(Dset_flavin.cellID) & set(Dset_mCherry_unsliced.cellID))
    Dset_mCherry = \
            Dset_mCherry_unsliced.loc[Dset_mCherry_unsliced.cellID.isin(idx_both)]

    # Arranges information into DatasetAttr objects
    # dummy -- will be better when I re-structure things... am just re-using a 
    # function for quick-and-dirty purposes, and it's obviously redundant
    mCherry_data = pipeline.dataimport.CellAttr_from_datasets( \
            timeseries_df = Dset_mCherry,
            categories_array = Dset_dcategory,
            births_df = Dset_births,
            sampling_pd = sampling_period)
    mCherry = pipeline.DatasetAttr(mCherry_data)
    mCherry_MATLABids = [cell.MATLABid for cell in mCherry.cells]

    # Add labels
    for ii, cell in enumerate(Dset.cells):
        cell.mCherry = pipeline.Fluo('mCherry')
        if cell.strain == 'htb2_mCherry_CRISPR':
            cell.mCherry.exposure = 100
        else:
            cell.mCherry.exposure = 0

        # loads in reading, cross-referencing by MATLABid.  This is awful, I know.
        if cell.MATLABid in mCherry_MATLABids:
            cell.mCherry.reading = \
                mCherry.cells[mCherry_MATLABids.index(cell.MATLABid)].y
except FileNotFoundError as error:
    print(error)
    print(f'No mCherry time series associated with this experiment: {filename_prefix}')

[Errno 2] No such file or directory: './data/arin/Omero20016_mCherry.csv'
No mCherry time series associated with this experiment: ./data/arin/Omero20016_


Define working dataset (list of cells)

In [4]:
Wlist = Dset.cells
#Wlist = [cell for cell in Dset.cells if cell.strain == 'swe1_Del']
len(Wlist)

1330

Chop up time series (exclude the end in which there is starvation)

In [5]:
interval_start = 0
interval_end = 168

for cell in Wlist:
    cell.time = cell.time[interval_start:interval_end]
    cell.flavin.reading = cell.flavin.reading[interval_start:interval_end]

Remove cells than have NaNs.  AR doesn't like it.

In [6]:
Wlist = [cell for cell in Wlist if not np.isnan(cell.flavin.reading).any()]
len(Wlist)

668

In [7]:
from collections import Counter
count_strain = Counter([cell.strain for cell in Wlist])
print(count_strain)

Counter({'zwf1_Del': 446, 'by4741': 222})


Sandbox: local regression

In [50]:
timeseries = Wlist[502].flavin.reading
plt.plot(timeseries)
plt.show()

from scipy.signal import savgol_filter
timeseries_sg = savgol_filter(timeseries, window_length=15, polyorder=5)
plt.plot(timeseries_sg)
plt.show()

from statsmodels.tsa.seasonal import STL
# Set period and seasonal according to the pd found by AR.  This could be a first approximation.
# Otherwise, set it at 15 (75 min, a knowledge-informed first approximation)
# But there's going to be an issue if I switch to a slower one e.g. zwf1...
# Another approach is setting it at a low value, e.g. 7 and apply the same value to all time series.
stl = STL(timeseries, period=25, seasonal=25, robust=False)
res = stl.fit()
fig = res.plot()

Add spectra

In [51]:
for cell in Wlist:
    cell.flavin.reading_processed = cell.flavin.reading
    #add_classicalAttr(cell, oversampling_factor = 1)
    add_autoregAttr(cell)
    #print(cell.cellid)

AR orders in whole dataset, by strain -- represented as histograms

In [52]:
import pandas as pd

list_positions = [list_position
                  for (list_position, cell) in enumerate(Wlist)]

ar_orders = pd.DataFrame(data = {
    'list_position': list_positions,
    'strain': [Wlist[list_position].strain for list_position in list_positions],
    'order': [Wlist[list_position].flavin.autoreg.order for list_position in list_positions],
})

strain_list = set(ar_orders.strain)

for strain in strain_list:
    fig, ax = plt.subplots()
    plt.hist(
        ar_orders.loc[ar_orders.strain == strain].order,
        bins = np.arange(22))
    plt.xticks(np.arange(22))
    plt.title(strain)
    plt.xlabel('order')
    plt.ylabel('count')

Compute period of 'smoothed periodogram', if appropriate

In [53]:
for cell in Wlist:
    cell.flavin.autoreg.add_pd()

/home/arin/git/time-series-pipeline/pipeline/__init__.py:60: RuntimeWarning: divide by zero encountered in true_divide
  self.pd = (1/self.freqs[self.power == max(self.power)])[0]


In [54]:
import pandas as pd

list_positions = [list_position
                  for (list_position, cell) in enumerate(Wlist)
                  if np.isfinite(cell.flavin.autoreg.pd)]

oscillating_cells = pd.DataFrame(data = {
    'list_position': list_positions,
    'strain': [Wlist[list_position].strain for list_position in list_positions],
    'period': [Wlist[list_position].flavin.autoreg.pd for list_position in list_positions],
    'max_power': [max(Wlist[list_position].flavin.autoreg.power) for list_position in list_positions],
    'order': [Wlist[list_position].flavin.autoreg.order for list_position in list_positions],
})

print(oscillating_cells)

    list_position    strain       period  max_power  order
0              60    by4741   324.350649   1.033094      5
1              84    by4741    76.259542   1.291732     17
2              88    by4741   396.428571   2.760693     18
3             113    by4741   846.610169   1.243889     19
4             118    by4741   423.305085   3.384517     19
5             129    by4741  1611.290323   1.025469     17
6             168    by4741   205.555556   1.633464     11
7             185    by4741   832.500000   1.126909     16
8             210    by4741   117.806604   1.895472      5
9             248  zwf1_Del   177.127660   1.342261     12
10            253  zwf1_Del   132.142857   7.870039      9
11            254  zwf1_Del  1427.142857   1.000146      7
12            263  zwf1_Del   306.441718   1.039466      5
13            265  zwf1_Del   247.277228   1.061638      5
14            268  zwf1_Del   129.069767   3.444091      7
15            270  zwf1_Del   225.000000   4.965026     

In [55]:
list_position = 502

Wlist[list_position].plot_ts()
Wlist[list_position].flavin.plot_ps(pdgram='autoreg', pd=False)
from scipy.signal import find_peaks
peaks, _ = find_peaks(Wlist[list_position].flavin.autoreg.power)
print(Wlist[list_position].flavin.autoreg.freqs[peaks])
print(1/np.array(Wlist[list_position].flavin.autoreg.freqs[peaks]))

[0.0029029]
[344.48275862]


White noise

Plotting one

In [93]:
timeaxis = Wlist[0].time
timeseries = np.random.normal(0, 1, len(timeaxis))

from scipy.signal import find_peaks
from pipeline.ar_grima2020 import AR_Fit, AR_Power, optimise_ar_order

# Model TS
optimal_ar_order = optimise_ar_order(timeseries, int(3*np.sqrt(len(timeseries))))
print(optimal_ar_order)
model = AR_Fit(timeseries, optimal_ar_order)
timeseries_modelled = np.empty(model.length)
for index in range(model.length):
    if index < optimal_ar_order:
        timeseries_modelled[index] = timeseries[index]
    else:
        preceding_points = timeseries[index-optimal_ar_order:index]
        linear_combination = np.dot(model.ar_coeffs[1::], preceding_points[::-1])
        timeseries_modelled[index] = linear_combination
        
# Plot time series
fig, ax = plt.subplots()
fig.set_size_inches((10,4))
ax.plot(timeaxis, timeseries, '#b785d5', label = 'White noise')
ax.plot(timeaxis, timeseries_modelled, '#430467', label = 'Autoregressive model')
ax.set_xlim([0,840])
ax.set_xticks(np.linspace(0,800,9))
ax.legend()
plt.xlabel('Time (min)')
plt.ylabel('Fluorescence, zero-centred (AU)')

# Plot periodogram
freq_npoints = 1000
order, freqs, power = pipeline.periodogram.autoreg(timeaxis, timeseries, freq_npoints)
peak_indices, _ = find_peaks(power)
peak_locs = freqs[peak_indices]

fig, ax = plt.subplots()
ax.plot(freqs, power, '#430467')
for peak_index in peak_indices:
    ax.axvline(freqs[peak_index], ymin = 0, ymax = power[peak_index],
               color = '#6f0aaa', linestyle = ':')
#ax.set_xlim([0,0.02])
ax.set_xticks(np.linspace(0,0.02,5))
#ax.set_ylim([0,14])
ax.set_xlabel('Frequency ($min^{-1}$)')
ax.set_ylabel('Power (dimensionless)')
ax.set_title('Autoregressive Model-Based Periodogram')
plt.show()

1


Hacking

In [113]:
import pandas as pd
from pipeline.ar_grima2020 import AR_Fit, AR_Power, optimise_ar_order

iterations = 10000

timeaxis = Wlist[0].time
orders = np.empty(iterations)
for iteration in range(iterations):
    timeseries = np.random.normal(0, 1, len(timeaxis))
    optimal_ar_order = optimise_ar_order(timeseries, int(3*np.sqrt(len(timeseries))))
    orders[iteration] = optimal_ar_order
    
freq_table = pd.Series(orders).value_counts().sort_index()
print(freq_table)

1.0     7218
2.0     1114
3.0      528
4.0      365
5.0      246
6.0      147
7.0      122
8.0       66
9.0       64
10.0      35
11.0      30
12.0      17
13.0      15
14.0      10
15.0       9
16.0       1
17.0       7
19.0       3
20.0       1
23.0       1
26.0       1
dtype: int64


In [117]:
plt.hist(orders, bins = np.arange(22))
plt.xlabel('order')
plt.ylabel('count')
plt.title('white noise (n = 10,000)')

Text(0.5, 1.0, 'white noise (n = 10,000)')

PROBLEM: there's only one swe1Δ cell that the AR identifies as oscillating.  Changes definitely need to be made to the algorithm.  Perhaps this is where the model selection comes in, but there's _no way_ I'll be able to explore this in time for the conference.

# For poster

Causton - tsa1 tsa2

In [165]:
Wlist[264].births

array([  41.75      ,   82.85      ,  152.83333333,  202.85      ,
        247.88333333,  337.88333333,  347.86666667,  422.88333333,
        492.86666667,  502.85      ,  577.88333333,  592.9       ,
        652.88333333,  707.9       ,  737.91666667,  812.93333333,
        857.88333333,  892.93333333, 1017.93333333])

In [177]:
from pipeline.ar_grima2020 import AR_Fit, AR_Power, optimise_ar_order

# Inputs
births = np.array([41.75, 82.85, 152.83, 202.85, 247.88, 347.87, 422.88, 502.85, 577.88, 652.88, 707.9, 737.92, 812.93])
timeaxis = Wlist[264].time
timeseries = Wlist[264].flavin.reading - np.mean(Wlist[264].flavin.reading)

# Model TS
optimal_ar_order = optimise_ar_order(timeseries, int(3*np.sqrt(len(timeseries))))
print(optimal_ar_order)
model = AR_Fit(timeseries, optimal_ar_order)
timeseries_modelled = np.empty(model.length)
for index in range(model.length):
    if index < optimal_ar_order:
        timeseries_modelled[index] = timeseries[index]
    else:
        preceding_points = timeseries[index-optimal_ar_order:index]
        linear_combination = np.dot(model.ar_coeffs[1::], preceding_points[::-1])
        timeseries_modelled[index] = linear_combination

18


Text(0, 0.5, 'Fluorescence, zero-centred (AU)')

In [201]:
# Plot time series
fig, ax = plt.subplots()
fig.set_size_inches((10,4))
ax.plot(timeaxis, timeseries, '#b785d5', label = 'Biological time series')
ax.plot(timeaxis, timeseries_modelled, '#430467', label = 'Autoregressive model')
for birth_count, birth in enumerate(births):
    if birth_count == 0:
        ax.axvline(birth, ymin = 0, ymax = 1, color = '#6f0aaa', linestyle = '--', label = 'Birth event')
    else:
        ax.axvline(birth, ymin = 0, ymax = 1, color = '#6f0aaa', linestyle = '--')
ax.set_xlim([0,840])
ax.set_xticks(np.linspace(0,800,9))
ax.legend()
plt.title('Autoregressive model overlaid on biological time series')
plt.title('tsa1Δ tsa2Δ')
plt.xlabel('Time (min)')
plt.ylabel('Fluorescence, zero-centred (AU)')

Text(0, 0.5, 'Fluorescence, zero-centred (AU)')

In [209]:
# Plot periodogram
freqs = Wlist[264].flavin.autoreg.freqs
power = Wlist[264].flavin.autoreg.power
peak_indices, _ = find_peaks(Wlist[264].flavin.autoreg.power)
peak_locs = freqs[peak_indices]

fig, ax = plt.subplots()
ax.plot(freqs, power, '#430467')
for peak_index in peak_indices:
    ax.axvline(freqs[peak_index], ymin = 0, ymax = power[peak_index],
               color = '#6f0aaa', linestyle = ':')
ax.set_xlim([0,0.02])
ax.set_xticks(np.linspace(0,0.02,5))
#ax.set_ylim([0,14])
ax.set_xlabel('Frequency ($min^{-1}$)')
ax.set_ylabel('Power (dimensionless)')
ax.set_title('Autoregressive Model-Based Periodogram')
plt.show()